In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
### identify the gesture of the rock paper and scissor gesture

import kagglehub

# Download latest version
path = kagglehub.dataset_download("josiagiven/neural-network")

print("Path to dataset files:", path)

100%|██████████| 228M/228M [00:02<00:00, 96.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/josiagiven/neural-network/versions/1


In [ ]:
import shutil

shutil.move(path, "/content")

'/content/1'

In [ ]:
path = "/content/1/suit/images/"
path_train = path+"train/"
path_val = path+"val/"

from tensorflow.keras.preprocessing.image import ImageDataGenerator

training_datagen = ImageDataGenerator(
 rescale = 1./255,
 rotation_range=40,
 width_shift_range=0.2,
 height_shift_range=0.2,
 shear_range=0.2,
 zoom_range=0.2,
 horizontal_flip=True,
 fill_mode='nearest'
)

In [ ]:
train_generator = training_datagen.flow_from_directory(
 path_train,
 target_size=(150,150),
 class_mode='categorical'
)

Found 2296 images belonging to 3 classes.


In [ ]:
val_generator = training_datagen.flow_from_directory(
 path_val,
 target_size=(150,150),
 class_mode='categorical'
)

Found 943 images belonging to 3 classes.


In [ ]:
model = tf.keras.models.Sequential([
 # Note the input shape is the desired size of the image:
 # 150x150 with 3 bytes color
 # This is the first convolution
 tf.keras.layers.Conv2D(64, (3,3), activation='relu',
 input_shape=(150, 150, 3)),
 tf.keras.layers.MaxPooling2D(2, 2),
 # The second convolution
 tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
 tf.keras.layers.MaxPooling2D(2,2),
 # The third convolution
 tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
 tf.keras.layers.MaxPooling2D(2,2),
 # The fourth convolution
 tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
 tf.keras.layers.MaxPooling2D(2,2),
 # Flatten the results to feed into a DNN
 tf.keras.layers.Flatten(),
 # 512 neuron hidden layer
 tf.keras.layers.Dense(512, activation='relu'),
 tf.keras.layers.Dense(3, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop',
 metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs=5,
 validation_data = val_generator, verbose = 1)

Epoch 1/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 255s 4s/step - accuracy: 0.7582 - loss: 0.6127 - val_accuracy: 0.8335 - val_loss: 0.4073
Epoch 2/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 253s 3s/step - accuracy: 0.8225 - loss: 0.4507 - val_accuracy: 0.8770 - val_loss: 0.3402
Epoch 3/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 263s 3s/step - accuracy: 0.8374 - loss: 0.4042 - val_accuracy: 0.9183 - val_loss: 0.2850
Epoch 4/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 259s 3s/step - accuracy: 0.8747 - loss: 0.3238 - val_accuracy: 0.9120 - val_loss: 0.2285
Epoch 5/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 274s 4s/step - accuracy: 0.9062 - loss: 0.2888 - val_accuracy: 0.9618 - val_loss: 0.1511


In [ ]:
from google.colab import files
from keras.preprocessing import image
uploaded = files.upload()
for fn in uploaded.keys():
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)

Saving images.jpeg to images.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
images.jpeg
[[1.0000000e+00 0.0000000e+00 5.1159645e-08]]


In [ ]:
# Save as .h5 file
model.save("/content/rps_model.h5")

# Create a directory to upload
import os
os.makedirs("/content/rps_model_dir", exist_ok=True)
os.rename("/content/rps_model.h5", "/content/rps_model_dir/rps_model.h5")

In [ ]:
from huggingface_hub import login
login(token="")

In [1]:
from tensorflow.keras.models import save_model
from huggingface_hub import HfApi, HfFolder

# Upload the model manually
from huggingface_hub import HfApi

api = HfApi()

try:
  repo_url = api.create_repo(repo_id="navidfalah/Rock-paper-scissors-model", private=False)
except:
  pass

api.upload_folder(
    folder_path="/content/rps_model_dir",
    repo_id="navidfalah/Rock-paper-scissors-model"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ValueError: Provided path: '/content/rps_model_dir' is not a directory